# Temporal filtering

A bandpass filter in resting-state fMRI is used to isolate specific frequency components of the BOLD signal, typically within the range of 0.01–0.1 Hz. This filtering process helps remove high-frequency noise and low-frequency drift, improving the detection of meaningful neural fluctuations related to brain connectivity. It ensures that only relevant temporal patterns are retained for further analysis.


-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------


## band-pass filter (0.01-0.1 Hz) resting-state


In [ ]:
import glob
import os
import subprocess

data_path = '/mnt/d/Data/Human/ObjectCategorization/analysis2'
bold_dirs = glob.glob('%s/sub-*/ses-*/func/rest*/*denoised.nii.gz'%(data_path))
LowPass = False
HighPass = False
AllPass = False
BandPass = True

hp = 0.01 # 99999 means don't apply high pass filter. 0 means only apply low pass filter.
lp = 0.1  # 99999 means don't  apply low pass filter. 

for curr_bold in bold_dirs:
    sub_dir = os.path.dirname(curr_bold)
    
    print('Currect file is: %s'%(curr_bold))

    # change directory to the res_dir 
    print('\t 1) Changing the working directory to %s'%(sub_dir))
    os.chdir(sub_dir)
    
    
    
     # If neither lowpass or highpass is set, do an allpass filter (fbot=0 ftop=99999)
   # If ONLY highpass is set, do a highpass filter (fbot=${hp} ftop=99999)
   # If ONLY lowpass is set, do a lowpass filter (fbot=0 ftop=${hp})
   # If both lowpass and highpass are set, do a bandpass filter (fbot=${hp} ftop=${lp})
    if AllPass:
        # allpass filter
        fbot=0
        ftop=99999
        #hp=0
        #lp=99999
        #filtType="AllPass"
        print("\t 2) Performing an 'allpass' filter.  Removal of '0' and Nyquist only.")
    elif HighPass:
        # highpass filter
        fbot=hp
        ftop=99999
        #lp=99999
        #filtType="HighPass"
        print("\t 2) Performing a 'HighPass' filter.  Frequencies below %s will be filtered."%(hp))
    elif LowPass:
        # lowpass filter
        fbot=0
        ftop=lp
        #hp=0
        #filtType="LowPass"
        print("\t 2) Performing a 'LowPass' filter.  Frequencies above %s will be filtered."%(lp))
    elif BandPass:
        # bandpass filter (low and high)
        fbot=hp
        ftop=lp
        #filtType="BandPass"
        print("\t 2) Performing a 'BandPass' filter.  Frequencies between %s & %s will be filtered."%(lp, hp))
        
    os.system('3dTproject -input %s -prefix func_bandpassed.nii.gz -mask reg/example_func_brain_mask.nii.gz -bandpass %s %s -verb'%(curr_bold, fbot, ftop)) # to remove motion, csf, weight matter and global signal you can add -ort "${regressorsFile}" 
    
    # add mean back in
    print('\t 3) Calculating the mean of denoised functional data ... ')
    os.system('3dTstat -mean -prefix func_mean.nii.gz %s'%(curr_bold))
    
    print('\t 4) Adding mean to filtered data, usefull for presentation')
    os.system('3dcalc -a func_bandpassed.nii.gz -b func_mean.nii.gz -expr "a+b" -prefix bold_mcf_st_smoothed_intnorm_denoised_bandpassed.nii.gz')
    os.system('rm -r *.1D func*')
